In [1]:
import sys
sys.path.append('..')
import os
import sklearn.metrics.cluster
import sklearn.cluster
import seaborn as sns
import scanpy as sc
import sklearn as sk
from sklearn.neighbors import NearestNeighbors
import matplotlib as mpl
from matplotlib import pyplot as plt
import skdim
import seaborn as sns
import time
import numpy as np
import pandas as pd
import pickle
import rpy2.robjects as ro
import rpy2.robjects.numpy2ri
import rpy2.robjects.packages as rpackages
import utils
import anndata2ri
import gc
import benchmark_common_functions
from itertools import product

anndata2ri.activate()
ro.numpy2ri.activate()
readRDS = ro.r['readRDS']
%load_ext rpy2.ipython

In [2]:
rootdir = '../data/DuoClustering2018/sce_filteredM3Drop10/'
resdir = '../results/DuoClustering2018/sce_filteredM3Drop10/'

fnames = os.listdir(rootdir)
fnames = [f for f in fnames if 'sce' in f and '.rds' in f]

#get_res_path = lambda fname: f'{resdir}{fname}_norm{do_norm}_log{do_log}_pca{do_pca}_ncomps{n_comps}_{metric}_weighted{weighted}_knn{n_neighbors}_seed{seed}.pkl'
#get_res_path_orig = lambda fname: f'{resdir}{fname}_orig_pca{do_pca}_ncomps{n_comps}_{metric}_weighted{weighted}_knn{n_neighbors}_seed{seed}.pkl'
get_res_path = lambda fname: f'{resdir}{fname}_norm{do_norm}_scale{norm_scale}_log{do_log}_pca{do_pca}_ncomps{n_comps}_{metric}_weighted{weighted}_knn_sqrt_seed{seed}.pkl'
get_res_path_orig = lambda fname: f'{resdir}{fname}_orig_pca{do_pca}_ncomps{n_comps}_{metric}_weighted{weighted}_knn_sqrt_seed{seed}.pkl'

# Run all datasets original

In [3]:
#fixed params
do_norm = False
norm_scale= False
do_log = False
do_pca = True
#n_neighbors = 10
seed = 0

#vary params
metric=('cosine','euclidean')
n_comps=(25,50,100,500)
weighted=(True,)

params_list = list(product(metric,n_comps,weighted))

In [4]:
for metric,n_comps,weighted in params_list:
    print(metric,n_comps,weighted)
    for fname in fnames:
        if get_res_path_orig(fname[:-4]).split('/')[-1] in os.listdir(resdir):
            continue
        print('\t\t'+fname)
        adata = readRDS(rootdir+fname)
        adata.X = adata.layers['logcounts']
            
        results_dict = benchmark_common_functions.clustering_analysis(
            adata,
            true_labels = adata.obs['phenoid'],
            do_norm = do_norm,
            norm_scale = norm_scale,
            do_log = do_log,
            do_pca = do_pca,
            n_neighbors=int(round(np.sqrt(len(adata)))),
            n_clusters= adata.obs['phenoid'].nunique(),
            metric=metric,
            weighted=weighted,  #weighted adjmat for louvain/leiden clustering ?
            seed=seed,
            n_comps=n_comps
        )

        with open(get_res_path_orig(fname[:-4]),'wb') as f:
            pickle.dump(results_dict,f)
        %xdel adata
        gc.collect()

cosine 25 True
cosine 50 True
cosine 100 True
cosine 500 True
euclidean 25 True
euclidean 50 True
euclidean 100 True
		sce_filteredM3Drop10_Zhengmix8eq.rds
			Preprocessing done: 0.02 mn
			Inputs generated: 0.05 mn
			Scanpy louvain/leiden clus: 0.17 mn
			Louvain/leiden clus: 0.01 mn
			Scanpy louvain/leiden clus, searching ground truth: 0.27 mn
			Base louvain/leiden clus, searching ground truth: 0.02 mn
			Scoring: 0.0 mn
			Hubness methods full pipeline: 0.48 mn
euclidean 500 True
		sce_filteredM3Drop10_Koh.rds
			Preprocessing done: 0.02 mn
			Inputs generated: 0.01 mn
			Scanpy louvain/leiden clus: 0.01 mn
			Louvain/leiden clus: 0.0 mn
			Scanpy louvain/leiden clus, searching ground truth: 0.02 mn
			Base louvain/leiden clus, searching ground truth: 0.0 mn
			Scoring: 0.0 mn
			Hubness methods full pipeline: 0.13 mn
		sce_filteredM3Drop10_KohTCC.rds
			Preprocessing done: 0.34 mn
			Inputs generated: 0.02 mn
			Scanpy louvain/leiden clus: 0.01 mn
			Louvain/leiden clus: 0.0 mn


R[write to console]: Loading required package: splatter



			Preprocessing done: 0.02 mn
			Inputs generated: 0.01 mn
			Scanpy louvain/leiden clus: 0.02 mn
			Louvain/leiden clus: 0.0 mn
			Scanpy louvain/leiden clus, searching ground truth: 0.02 mn
			Base louvain/leiden clus, searching ground truth: 0.0 mn
			Scoring: 0.0 mn
			Hubness methods full pipeline: 0.1 mn
		sce_filteredM3Drop10_SimKumar4hard.rds
			Preprocessing done: 0.02 mn
			Inputs generated: 0.01 mn
			Scanpy louvain/leiden clus: 0.01 mn
			Louvain/leiden clus: 0.0 mn
			Scanpy louvain/leiden clus, searching ground truth: 0.03 mn
			Base louvain/leiden clus, searching ground truth: 0.0 mn
			Scoring: 0.0 mn
			Hubness methods full pipeline: 0.11 mn
		sce_filteredM3Drop10_SimKumar8hard.rds
			Preprocessing done: 0.02 mn
			Inputs generated: 0.01 mn
			Scanpy louvain/leiden clus: 0.01 mn
			Louvain/leiden clus: 0.0 mn
			Scanpy louvain/leiden clus, searching ground truth: 0.04 mn
			Base louvain/leiden clus, searching ground truth: 0.0 mn
			Scoring: 0.0 mn
			Hubness methods 

# Run all datasets

In [3]:
#fixed params
do_norm = False
norm_scale = False
do_log = True
do_pca = True
n_neighbors = 10
seed = 0

#vary params
metric=('cosine','euclidean')
n_comps=(25,50,100,500)
weighted=(True,False)
do_norm = ('duo','seurat','zheng17',None)
norm_scale = (True,False)
params_list = list(product(metric,n_comps,weighted,do_norm,norm_scale))

In [4]:
for metric,n_comps,weighted,do_norm,norm_scale in params_list:
    print(metric,n_comps,weighted,do_norm,norm_scale)
    for fname in fnames:
        if get_res_path(fname[:-4]).split('/')[-1] in os.listdir(resdir):
            continue
        print('\t\t'+fname)
        adata = readRDS(rootdir+fname)
        if 'Sim' in fname:
            adata.X = adata.layers['counts']
        results_dict = benchmark_common_functions.clustering_analysis(
            adata,
            true_labels = adata.obs['phenoid'],
            do_norm = do_norm,
            norm_scale = norm_scale,
            do_log = do_log,
            do_pca = do_pca,
            n_neighbors=n_neighbors,
            n_clusters= adata.obs['phenoid'].nunique(),
            metric=metric,
            weighted=weighted,  #weighted adjmat for louvain/leiden clustering ?
            seed=seed,
            n_comps=n_comps
        )

        with open(get_res_path(fname[:-4]),'wb') as f:
            pickle.dump(results_dict,f)
    
        %xdel adata
        gc.collect()

cosine 25 True duo True
		sce_filteredM3Drop10_Koh.rds
		duo norm retained 4898 genes
			Preprocessing done: 0.01 mn
			Inputs generated: 0.01 mn
			Scanpy louvain/leiden clus: 0.16 mn
			Louvain/leiden clus: 0.0 mn
			Scanpy louvain/leiden clus, searching ground truth: 0.02 mn
			Base louvain/leiden clus, searching ground truth: 0.0 mn
			Spectral clus: 0.02 mn
			Scoring: 0.01 mn
			Hubness methods full pipeline: 0.13 mn
		sce_filteredM3Drop10_KohTCC.rds
		duo norm retained 5000 genes
			Preprocessing done: 0.12 mn
			Inputs generated: 0.0 mn
			Scanpy louvain/leiden clus: 0.01 mn
			Louvain/leiden clus: 0.0 mn
			Scanpy louvain/leiden clus, searching ground truth: 0.01 mn
			Base louvain/leiden clus, searching ground truth: 0.0 mn
			Spectral clus: 0.01 mn
			Scoring: 0.0 mn
			Hubness methods full pipeline: 0.05 mn
		sce_filteredM3Drop10_Kumar.rds
		duo norm retained 4516 genes
			Preprocessing done: 0.02 mn
			Inputs generated: 0.0 mn
			Scanpy louvain/leiden clus: 0.0 mn
			Louva

R[write to console]: Loading required package: splatter



		duo norm retained 4361 genes
			Preprocessing done: 0.04 mn
			Inputs generated: 0.01 mn
			Scanpy louvain/leiden clus: 0.01 mn
			Louvain/leiden clus: 0.0 mn
			Scanpy louvain/leiden clus, searching ground truth: 0.02 mn
			Base louvain/leiden clus, searching ground truth: 0.0 mn
			Spectral clus: 0.0 mn
			Scoring: 0.0 mn
			Hubness methods full pipeline: 0.05 mn
		sce_filteredM3Drop10_SimKumar4hard.rds
		duo norm retained 4364 genes
			Preprocessing done: 0.01 mn
			Inputs generated: 0.0 mn
			Scanpy louvain/leiden clus: 0.01 mn
			Louvain/leiden clus: 0.0 mn
			Scanpy louvain/leiden clus, searching ground truth: 0.03 mn
			Base louvain/leiden clus, searching ground truth: 0.0 mn
			Spectral clus: 0.01 mn
			Scoring: 0.01 mn
			Hubness methods full pipeline: 0.1 mn
		sce_filteredM3Drop10_SimKumar8hard.rds
		duo norm retained 4360 genes
			Preprocessing done: 0.02 mn
			Inputs generated: 0.0 mn
			Scanpy louvain/leiden clus: 0.01 mn
			Louvain/leiden clus: 0.0 mn
			Scanpy louvain/

TypeError: ufunc 'log1p' output (typecode 'd') could not be coerced to provided output parameter (typecode 'i') according to the casting rule ''same_kind''